In [ ]:
from loadData import sentencesDataset, GenerateWordEmbeddings

In [ ]:
rawSentences = sentencesDataset("../data/yelp/sentiment.train.0", "../data/yelp/")
print(rawSentences[0])

# dimension of word embeddings set to 200 per Zhao et al.
yelpData = sentencesDataset("../data/yelp/sentiment.train.0", "../data/yelp/", 
                            transform=GenerateWordEmbeddings(200, "../data/glove.6B/"))  
print(yelpData[0])

In [ ]:
print(len(yelpData[0][0]))
    